In [79]:
denominations={1:10, 5:0, 10:2, 20:3}
def withdraw(withdrawal, denominations):
    valid_denom=[1,5,10,20]
    
    #sort the denominations from large to small 
    denom_sort = sorted(denominations.keys(), reverse=True)  #do this as we are checking from largest if we do not have a denominations less than this current we can take from this large denomination
    withdrawn = { denom:0 for denom in denominations}
    
    # if denominations not in the teller raise valueerror
    if not all(denom in valid_denom for denom in withdrawal.keys()):
        raise ValueError("Teller only supports this {} denomination".format(denominations.key()))

    for denom, quantity in withdrawal.items():
        while quantity>0:
            '''
            if the teller has the quantity add from the minimum of requested and actual quantity in teller of respective denomination to satisfy atleast 
            a part of the withdrawal quantity.
            Subtract the added quantity to the withdrawal from the teller(denominations).
            '''
            if denom in denominations and denominations[denom]>0:
                wd_qty = min(quantity, denominations[denom])
                withdrawn[denom] += wd_qty
                denominations[denom] -= wd_qty
                quantity -= wd_qty
            else:
                # Break larger denominations if available
                for large_denom in denom_sort:
                    if large_denom > denom and denominations[large_denom] > 0:
                        # Break one large denomination into smaller denominations
                        denominations[large_denom] -= 1
                        break_down_qty = large_denom // denom
                        denominations[denom] += break_down_qty

                        # Adjust leftover notes from breakdown
                        leftover = large_denom % denom
                        if leftover > 0 and leftover in denominations:
                            denominations[leftover] += 1
                        break
                else:
                    # Return failure if no larger denominations can satisfy the request
                    return False, denominations

    for denom, qty in withdrawal.items():
        if withdrawn[denom] < qty:
            return False, denominations

    return withdrawn, denominations


denominations = {1: 3, 5: 0, 10: 1, 20: 2}
requested = {1: 5, 10: 1}
try:
    withdrawn, updated_denominations = withdraw(requested, denominations)
    print("Withdrawal Result:", withdrawn)
    print("Updated Denominations:", updated_denominations)
except ValueError as e:
    print("Error:", e)
                        

Withdrawal Result: {1: 5, 5: 0, 10: 1, 20: 0}
Updated Denominations: {1: 18, 5: 0, 10: 0, 20: 1}


In [45]:
def withdraw(withdrawal, denominations):
    """
    Withdraw the requested denominations from the teller.

    Parameters:
        withdrawal (dict): Requested denominations with quantities.
        denominations (dict): Available denominations in the teller.

    Returns:
        tuple: Withdrawn denominations and updated teller denominations if successful.
               False and unchanged denominations if withdrawal cannot be satisfied.
    """
    valid_denom = [1, 5, 10, 20]
    withdrawn = {denom: 0 for denom in valid_denom}  # To track what was withdrawn

    # Validate the input
    if not all(denom in valid_denom for denom in withdrawal.keys()):
        raise ValueError("Teller only supports the following denominations: {}".format(valid_denom))

    # Process each requested denomination
    for denom, qty_needed in withdrawal.items():
        while qty_needed > 0:
            if denominations.get(denom, 0) > 0:
                # Withdraw directly if enough notes of the denomination exist
                wd_qty = min(qty_needed, denominations[denom])
                withdrawn[denom] += wd_qty
                denominations[denom] -= wd_qty
                qty_needed -= wd_qty
            else:
                # Break down larger denominations if the current denomination is insufficient
                larger_broken = False
                for larger_denom in sorted(denominations.keys(), reverse=True):
                    if larger_denom > denom and denominations[larger_denom] > 0:
                        # Break one larger denomination
                        denominations[larger_denom] -= 1
                        breakdown_qty = larger_denom // denom
                        leftover = larger_denom % denom

                        # Add broken-down notes to the current denomination
                        denominations[denom] += breakdown_qty
                        if leftover > 0:
                            # Redistribute leftover to smaller denominations
                            for smaller_denom in sorted(valid_denom, reverse=True):
                                if leftover >= smaller_denom:
                                    smaller_qty = leftover // smaller_denom
                                    denominations[smaller_denom] += smaller_qty
                                    leftover -= smaller_qty * smaller_denom
                        larger_broken = True
                        break

                if not larger_broken:
                    # No larger denomination can be broken to satisfy the withdrawal
                    return False, denominations

    # Final verification of withdrawal
    for denom, qty in withdrawal.items():
        if withdrawn[denom] < qty:
            return False, denominations

    return withdrawn, denominations


# Example Usage
denominations = {1: 3, 5: 0, 10: 1, 20: 2}
requested = {1: 5, 10: 1}

try:
    withdrawn, updated_denominations = withdraw(requested, denominations)
    print("Withdrawal Result:", withdrawn)
    print("Updated Denominations:", updated_denominations)
except ValueError as e:
    print("Error:", e)

Withdrawal Result: {1: 5, 5: 0, 10: 1, 20: 0}
Updated Denominations: {1: 18, 5: 0, 10: 0, 20: 1}


In [ ]:
class LimitedTellerMachine:
    def __init__(self):
        self.denominations = {1: 0, 5: 0, 10: 0, 20: 0}

    def deposit(self, deposit_dict):
        for denom, quantity in deposit_dict.items():
            if denom not in self.denominations:
                raise ValueError(f"Unsupported denomination: {denom}")
            if quantity < 0:
                raise ValueError("Cannot deposit a negative quantity")
            self.denominations[denom] += quantity

    def withdraw(self, withdraw_dict):
        temp_denominations = self.denominations.copy()
        for denom in sorted(withdraw_dict.keys(), reverse=True):  # Start from highest denomination
            quantity = withdraw_dict[denom]
            if not self._withdraw_denomination(temp_denominations, denom, quantity):
                return False
        self.denominations = temp_denominations
        return True

    def _withdraw_denomination(self, temp_denominations, denom, quantity):
        if temp_denominations[denom] >= quantity:
            temp_denominations[denom] -= quantity
            return True
    
        remaining = quantity - temp_denominations[denom]
        temp_denominations[denom] = 0
    
        for higher_denom in sorted([d for d in self.denominations if d > denom]):
            while remaining > 0 and temp_denominations[higher_denom] > 0:
                temp_denominations[higher_denom] -= 1
                change = higher_denom
                while change > 0:
                    next_lower = max(d for d in self.denominations if d <= change)
                    if next_lower == denom:
                        temp_denominations[denom] += change // next_lower
                        remaining -= change // next_lower
                        break
                    else:
                        temp_denominations[next_lower] += change // next_lower
                    change = change % next_lower
    
            if remaining == 0:
                break
    
        return remaining == 0
         

    def get_quantity(self, denom):
        if denom not in self.denominations:
            raise ValueError(f"Unsupported denomination: {denom}")
        return self.denominations[denom]
        
# Initialize the teller machine and deposit initial amounts
teller = LimitedTellerMachine()
teller.deposit({1: 3, 5: 0, 10: 1, 20: 2})

# Perform the withdrawal
result = teller.withdraw({1: 5, 10: 1})

# Check the final state of the teller machine
final_state = {denom: teller.get_quantity(denom) for denom in [1, 5, 10, 20]}

print("Withdrawal successful:", result)
print("Final state:", final_state)